In [ ]:
pip install gradio


In [ ]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
default = pd.read_csv('https://github.com/ybifoundation/Dataset/raw/main/Credit%20Default.csv')


In [ ]:
default.head()       # Show first 5 rows
default.info()       # Data types and non-null values
default.describe()   # Summary statistics
default['Default'].value_counts()  # Count of 0s and 1s


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Income          2000 non-null   float64
 1   Age             2000 non-null   float64
 2   Loan            2000 non-null   float64
 3   Loan to Income  2000 non-null   float64
 4   Default         2000 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 78.3 KB


,count
Default,
0,1717
1,283


In [ ]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and prepare data
url = 'https://github.com/ybifoundation/Dataset/raw/main/Credit%20Default.csv'
default = pd.read_csv(url)

X = default.drop('Default', axis=1)
y = default['Default']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.7, random_state=2529)

model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

# Prediction and visual generation
def predict_with_visuals(income, age, loan):
    loan_to_income = loan / income
    input_data = pd.DataFrame([[income, age, loan, loan_to_income]],
                              columns=['Income', 'Age', 'Loan', 'Loan to Income'])
    input_scaled = scaler.transform(input_data)

    # Predict & get probability
    prediction = model.predict(input_scaled)[0]
    probability = model.predict_proba(input_scaled)[0][1]  # Probability of default

    result = "Default" if prediction == 1 else "No Default"
    confidence = round(probability * 100, 2)

    # Plotting
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))

    # Chart 1: Probability bar
    ax[0].bar(['No Default', 'Default'], model.predict_proba(input_scaled)[0], color=['green', 'red'])
    ax[0].set_title('Prediction Probability')
    ax[0].set_ylim(0, 1)

    # Chart 2: Loan to Income bar
    ax[1].bar(['Loan', 'Income'], [loan, income], color=['orange', 'blue'])
    ax[1].set_title(f'Loan to Income Ratio: {round(loan_to_income, 2)}')

    plt.tight_layout()

    return result + f"\n\nConfidence of Default: {confidence}%", fig

# Gradio interface
interface = gr.Interface(
    fn=predict_with_visuals,
    inputs=[
        gr.Number(label="Annual Income (₹)"),
        gr.Number(label="Age"),
        gr.Number(label="Loan Amount (₹)")
    ],
    outputs=[
        gr.Textbox(label="Prediction Result"),
        gr.Plot(label="Visual Summary")
    ],
    title="💳 Credit Card Default Predictor with Visuals",
    description="Enter income, age, and loan to predict credit card default. Visuals included!"
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://48992d1e90f75f05b5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
